# Домашнее задание № 9. Предобученные трансформеры 

In [1]:
!pip install sklearn pandas matplotlib transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.1 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=5fb83bd85755580acbafe8345d78d7456db36fd58fa22f7e1687300e55791070
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [2]:
# стандартные библиотеки
import os, re
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
%matplotlib inline


# tf и huggingface 
import tensorflow as tf

In [3]:
from transformers import BertTokenizer, TFBertModel

In [4]:
from transformers import AutoTokenizer, TFAutoModel

### Задание 1 (8 баллов).
Дообучите 3 предобученных модели на тех же данных, что и в семинаре. Сравните качество и выберите лучшую. 
Модели должны отличаться друг от друга не только параметрами ru/en/multilingual, base/large, cased/uncased. Все три выбранные модели должны работать лучше, чем бейзлайн без дообучения (т.е. если модель не работает, попробуйте другую).




In [5]:
data = pd.read_csv('lenta_sample.csv')
data.dropna(subset=['topic', 'text'], inplace=True)

In [6]:
# ФУНКЦИЯ ДЛЯ ПОДГОТОВКИ ДАННЫХ

def prepair_input(some_data, some_tokenizer):
  X = []

  for text in some_data.title:
    ids = some_tokenizer.encode(text)
    X.append(ids[:512])
  
  X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

  id2label = {i:label for i, label in enumerate(set(some_data.topic.values))}
  label2id = {l:i for i, l in id2label.items()}

  y = tf.keras.utils.to_categorical([label2id[label] for label in some_data.topic.values])

  train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=some_data.topic)

  X_train, y_train = X[train_index], y[train_index]
  X_valid, y_valid = X[valid_index], y[valid_index]

  return X_train, y_train, X_valid, y_valid, y

In [7]:
#@title 1. Oригинальный берт, дообучение без замораживания
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained('bert-base-cased', trainable=True)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [8]:
X_train, y_train, X_valid, y_valid, y = prepair_input(data, tokenizer)

In [9]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf_1 = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_clf_1.compile(tf.optimizers.Adam(learning_rate=2e-6,
                                    ), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [10]:
model_clf_1.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2,
         epochs=55)

Epoch 1/55


288/288 [==============================] - 118s 280ms/step - loss: 2.9095 - accuracy: 0.0764 - rec_prec: 0.0000e+00 - val_loss: 2.7360 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 2/55
288/288 [==============================] - 76s 266ms/step - loss: 2.8698 - accuracy: 0.0694 - rec_prec: 0.0000e+00 - val_loss: 2.7111 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 3/55
288/288 [==============================] - 77s 269ms/step - loss: 2.8484 - accuracy: 0.0781 - rec_prec: 0.0000e+00 - val_loss: 2.7127 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 4/55
288/288 [==============================] - 76s 265ms/step - loss: 2.8256 - accuracy: 0.0851 - rec_prec: 0.0000e+00 - val_loss: 2.6925 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 5/55
288/288 [==============================] - 77s 266ms/step - loss: 2.8082 - accuracy: 0.0920 - rec_prec: 0.0000e+00 - val_loss: 2.6904 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 6/55
288/288 [===============

In [11]:
macc1 = round(model_clf_1.history.history['accuracy'][-1], 3)

In [12]:
#@title 2. Мультиязычный берт, дообучение с замораживанием предобученной модели
tokenizer_mult = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')
model_mult = TFAutoModel.from_pretrained('bert-base-multilingual-uncased', trainable=False)

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
X_train, y_train, X_valid, y_valid, y = prepair_input(data, tokenizer_mult)

In [14]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_mult({"input_ids":input_word_ids})

drop = tf.keras.layers.Dropout(0.1)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf_2 = tf.keras.Model(inputs=input_word_ids, outputs=dense)

model_clf_2.compile(tf.optimizers.Adam(learning_rate=2e-3), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [15]:
model_clf_2.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=25,
         epochs=60)

Epoch 1/60
24/24 [==============================] - 44s 1s/step - loss: 2.8599 - accuracy: 0.0764 - rec_prec: 0.0000e+00 - val_loss: 2.7762 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 2/60
24/24 [==============================] - 27s 1s/step - loss: 2.8025 - accuracy: 0.0868 - rec_prec: 0.0000e+00 - val_loss: 2.7495 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 3/60
24/24 [==============================] - 27s 1s/step - loss: 2.8133 - accuracy: 0.0642 - rec_prec: 0.0000e+00 - val_loss: 2.7450 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 4/60
24/24 [==============================] - 27s 1s/step - loss: 2.8258 - accuracy: 0.0816 - rec_prec: 0.0017 - val_loss: 2.7364 - val_accuracy: 0.0645 - val_rec_prec: 0.0000e+00
Epoch 5/60
24/24 [==============================] - 27s 1s/step - loss: 2.7979 - accuracy: 0.0781 - rec_prec: 0.0000e+00 - val_loss: 2.7548 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 6/60
24/24 [==============================] - 27

In [16]:
macc2 = round(model_clf_2.history.history['accuracy'][-1], 3)

In [17]:
#@title 3. Rubert
tokenizer_ru = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")
model_bert_ru = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
X_train, y_train, X_valid, y_valid, y = prepair_input(data, tokenizer_ru)

In [19]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_bert_ru({"input_ids":input_word_ids})

drop = tf.keras.layers.Dropout(0.1)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf_3 = tf.keras.Model(inputs=input_word_ids, outputs=dense)

model_clf_3.compile(tf.optimizers.Adam(learning_rate=2e-3), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [20]:
model_clf_3.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2,
         epochs=25)

Epoch 1/25


288/288 [==============================] - 116s 285ms/step - loss: 3.5431 - accuracy: 0.0712 - rec_prec: 0.0000e+00 - val_loss: 3.1974 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 2/25
288/288 [==============================] - 75s 262ms/step - loss: 3.0554 - accuracy: 0.0642 - rec_prec: 0.0000e+00 - val_loss: 2.9128 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 3/25
288/288 [==============================] - 76s 265ms/step - loss: 2.8878 - accuracy: 0.0764 - rec_prec: 0.0000e+00 - val_loss: 2.7943 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 4/25
288/288 [==============================] - 76s 265ms/step - loss: 2.8763 - accuracy: 0.0573 - rec_prec: 0.0000e+00 - val_loss: 2.7554 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 5/25
288/288 [==============================] - 75s 261ms/step - loss: 2.8330 - accuracy: 0.0608 - rec_prec: 0.0000e+00 - val_loss: 2.7642 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 6/25
288/288 [===============

In [21]:
macc3 = round(model_clf_3.history.history['accuracy'][-1], 3)

In [22]:
#@title Сравнение результатов
list_of_models = ['bert-base-cased', 'bert-base-multilingual-uncased', 'Geotrend/bert-base-ru-cased']
list_of_macc = [macc1, macc2, macc3]

tabledata = []
for j in range(len(list_of_models)):
    tabledata.append([list_of_models[j], list_of_macc[j]])

In [23]:
# Данные отсортированы в порядке убывания по Accuracy. Лучшая модель окажется в 1ой строчке

pd.DataFrame(tabledata, columns=['model','Accuracy metric']).sort_values(by = 'Accuracy metric',ascending=False)

,model,Accuracy metric
0,bert-base-cased,0.925
1,bert-base-multilingual-uncased,0.089
2,Geotrend/bert-base-ru-cased,0.071


### Задание  2 (2 балла)

Ответьте на следующие вопросы (напишите хотя бы по несколько предложений):

1) Чем ROBERTA отличается от BERT? В какой статье описана ROBERTA?

2) Чем модель T5 отличается от ROBERTA/BERT? 

**Ответ 1:** ROBERTA не предсказывает следующее предложение. В ней используется динамическое маскирование: новая маска генерируется каждый раз, когда модель видит предложение. В ней больше данных, чем в модели BERT (160 GB, примерно в 10 раз больше), и бОльшие батчи для обучения (по 8000 предложений); её обучение заняло больше времени (1 день на 1024 графических процессорах GPU). 

Статья: Liu, Y., Ott, M., Goyal, N., Du, J., Joshi, M., Chen, D., ... & Stoyanov, V. (2019). Roberta: A robustly optimized bert pretraining approach. arXiv preprint arXiv:1907.11692.

**Ответ 2:** T5 (Text-To-Text Transfer Transformer), то есть вход и выход модели представляются текстовыми строками. Т5 может решать две новые задачи: ответы на вопросы без предварительного обучения на специальном корпусе вопросов-ответов (Closed-book question answering) и генерация текста для заполнения пропусков переменной длины.